# Práctica 0 – MasterMind

En este _notebook_ irás implementando el juego [MasterMind](https://en.wikipedia.org/wiki/Mastermind_(board_game)) a partir de fragmentos de código que iremos presentando. El objetivo es practicar las construcciones básicas del lenguaje Swift.

## Funcionamiento

Un jugador (que en nuestro caso será el ordenador) crea una clave secreta seleccionando 4 fichas de color de entre un conjunto de fichas de 6 colores diferentes. Se admiten repeticiones: es válida una clave en que las 4 fichas sean de color rojo, por ejemplo. Hay otras variantes en las que hay más colores, o la clave consta de más fichas, o no se permiten repeticiones, etc; pero la más habitual es la que hemos descrito.

Otro jugador (un humano, en nuestro caso) debe adivinar la clave en un número máximo de intentos. Para ello, el juego progresa del siguiente modo:
* El jugador que debe adivinar la clave indica una combinación de colores.
* El ordenador indica los siguientes datos:
  - Cuántos de los colores indicados coinciden con los de la clave, en el mismo orden. Este será el numero de **aciertos**.
  - Cuántos de los colores indicados aparecen en la clave, pero en distinto orden. A este número le llamaremos de **semiaciertos**.
* Con esta información, el jugador elige otra combinación hasta que adivina la clave o se alcanza el número máximo de turnos.

Por ejemplo, dada la clave secreta 🔴🟡⚪🔴, la combinación 🔴🟡🔴🟡 tendría
* Dos aciertos: las dos primeras fichas coinciden.
* Un semiacierto: la tercera ficha de la combinación adivinada está en la clave, pero en otra posición.

Observa cómo las fichas que coinciden con exactitud se excluyen del cómputo de los semiaciertos. La cuarta ficha de la combinación propuesta, que es amarilla, no se considera un semiacierto, pues no aparece en ninguna de las otras dos fichas de la clave.

Para que te hagas una idea del programa que vamos a escribir, la siguiente imagen muestra un ejemplo de ejecución de una partida completa una vez terminada la implementación correctamente.

![mastermind_game](img/mastermind-game.png "Partida de MasterMind")

----

Para comenzar, vamos a aprovechar que Swift admite trabajar con cadenas Unicode, lo que nos permite representar caracteres emoji y otros símbolos:

In [1]:
print("🔴🟢🟡🔵⚫⚪")

🔴🟢🟡🔵⚫⚪


Para representar los posibles colores del juego, vamos a utilizar el siguiente tipo enumerado:

In [1]:
enum MasterMindColor {
    case red
    case green
    case yellow
    case blue
    case black
    case white
}

In [4]:
print(MasterMindColor.green)

green


Para que el juego sea lo más vistoso posible (dentro de las limitaciones del entorno Jupyter en el que nos encontramos), vamos a crear una propiedad que nos permita obtener la representación gráfica del color. Esto lo podemos añadir como una extensión del tipo `MasterMindColor`:

In [2]:
extension MasterMindColor {
    var emoji: String {
        switch self {
            case .red   : return "🔴"
            case .green : return "🟢"
            case .yellow: return "🟡"
            case .blue  : return "🔵"
            case .black : return "⚫"
            case .white : return "⚪"
        }
    }
}

Esta propiedad está ahora incluida en cualquier objeto de tipo `MasterMindColor`:

In [3]:
print(MasterMindColor.green.emoji) //.emoji hace ver las imagenes.

🟢


Mucho más bonito que lo anterior.

De igual modo, vamos a hacer una función que nos permita obtener un color de nuestro juego (es decir, un `MasterMindColor` a partir de un carácter que puede teclear el usuario. No todos los caracteres son válidos, así que nuestra función lanzará un error en caso de que le pidamos convertir un carácter que no tiene sentido:

In [3]:
enum MasterMindError: Error {
    case wrongCharacter    // The user supplied a wrong Character to `from`
}

extension MasterMindColor {
    static func from(emoji: Character) throws -> MasterMindColor {   //throw lanzar  
        switch emoji {
            case "🔴": return .red
            case "🟢": return .green
            case "🟡": return .yellow
            case "🔵": return .blue
            case "⚫": return .black
            case "⚪": return .white
            default: throw MasterMindError.wrongCharacter
        }
    }
}

In [12]:
print(MasterMindColor.from(emoji: "🟢"))

green


----

**Importante**: observa que la función `from(emoji:)` _no_ es una función de las instancias de `MasterMindColor`, sino una función del tipo. Esto es así porque la utilizamos para crear colores, no para transformar colores que ya hemos creado.

Para añadir funciones al tipo utilizamos la palabra **`static`**. Si no la hubiéramos puesto, la función estaría dentro de los objetos de ese tipo, como en el caso de la propiedad `emoji` que hemos definido unas celdas más arriba.

----

Ya puestos, vamos a hacer que `print` muestre también el color de forma gráfica. Para ello tenemos que adoptar el protocolo [`CustomStringConvertible`](https://developer.apple.com/documentation/swift/customstringconvertible) e implementar la propiedad `description`.

In [4]:
extension MasterMindColor: CustomStringConvertible {
    public var description: String { return emoji }
}

In [5]:
print(MasterMindColor.black)

⚫


Mucho mejor.

## Ejercicio 1

Nuestro juego va a pedir al usuario combinaciones de colores. El usuario podrá "teclear" los emojis correspondientes, pero es un rollo hacerlo así. Vamos a extender `MasterMindColor` con otra función que permita utilizar las letras `r, g, y, b, k, w` (`b` significa _blue_ y `k` significa _blac**k**_).

Se pide completar el fragmento de código siguiente e implementar la función `from(letter:)`, que debe funcionar del siguiente modo:
* Dado uno de los caracteres `r, g, y, b, k, w`, la función devolverá el color correspondiente. Por ejemplo, devolverá `.red` si la letra es la `r`.
* Se permite escribir las letras en mayúscula o en minúscula.
* Si la letra no es ninguna de las indicadas, `from(letter:)`, puede ser que sea uno de los emojis válidos. Para comprobarlo, `from(letter:)` invocará `from(emoji:)`, y devolverá el color correspondiente.
* Si la letra no es ninguna de las indicadas en este enunciado ni tampoco uno de los emojis que entiende `from(emoji:)`, se lanzará el error `MasterMindError.wrongCharacter`.

In [5]:
extension MasterMindColor {
    static func from(letter: Character) throws -> MasterMindColor {        
        
        switch letter {
            case "r": return .red
            case "g": return .green
            case "y": return .yellow
            case "b": return .blue
            case "k": return .black
            case "w": return .white
            case "R": return .red
            case "G": return .green
            case "Y": return .yellow
            case "B": return .blue
            case "K": return .black
            case "W": return .white
            case "🔴": return .red
            case "🟢": return .green
            case "🟡": return .yellow
            case "🔵": return .blue
            case "⚫": return .black
            case "⚪": return .white
            default: throw MasterMindError.wrongCharacter
        }
    }
}

Las siguientes dos celdas te ayudarán a comprobar si tu implementación es correcta. Si no escriben nada, es que todo ha ido bien. Si se escribe un mensaje de error, es que algo falla. Ten en cuenta que estos _tests_ sólo prueban algunos casos, no todos. Es tu responsabilidad probar exhaustivamente.

In [6]:
do {
    let black = try MasterMindColor.from(letter: "K")
    let blue = try MasterMindColor.from(letter: "b")
    if black != .black { print("Error al convertir 'K' al color negro.") }
    if blue != .blue { print("Error al convertir 'b' al color azul.")}
} catch {
    print("Se ha producido una excepción incorrecta")
}

In [8]:
do {
    try MasterMindColor.from(letter: "p")
    print("Error: la letra `p` no debería tener ningún color asociado, debería haberse lanzado una excepción.")
} catch {}

----

In [363]:
print(MasterMindColor.from(letter: "k"))

⚫


In [11]:
print(MasterMindColor.from(letter: "🟢"))

🟢


Ahora vamos a hacer otra función de apoyo que nos permita convertir un `String` con una combinación de colores, a un array `[MasterMindColor]`.

Observa que lo que queremos puede obtenerse fácilmente con `map`:

In [8]:
print("rbyK".map { try MasterMindColor.from(letter: $0) })

[🔴, 🔵, 🟡, ⚫]


Observa también cómo ha sido necesario poner `try` dentro del bloque de código de `map`. Es necesario puesto que `from(letter:)` puede lanzar una excepción.

En lugar de hacer el `map` siempre que lo necesitemos, vamos a hacer una extensión de `String` para mayor conveniencia. La siguiente función transforma una variable de tipo `String` en el array de colores asociado, siempre que sea posible. Si el `String` contiene caracteres que no representan colores, se lanzará una excepción:

In [9]:
extension String {
    func toMasterMindColorCombination() throws -> [MasterMindColor] {
        return try self.map { try MasterMindColor.from(letter: $0) }
    }
}

In [12]:
print("rgbk".toMasterMindColorCombination())

[🔴, 🟢, 🔵, ⚫]


In [9]:
let x = "rgbk" 
print(x.toMasterMindColorCombination())
print (x)

[🔴, 🟢, 🔵, ⚫]
rgbk


In [10]:
do {
    try print("kKGRy".toMasterMindColorCombination())
} catch {
    print("Error")
}

[⚫, ⚫, 🟢, 🔴, 🟡]


**Nota**: observa cuidadosamente los detalles de la implementación. Hemos tenido que poner `try` dentro del bloque que se ejecuta dentro de `map`, como hemos indicado más arriba. Pero también hemos tenido que ponerlo antes de llamar a `self.map`. Esto es así porque si el bloque lanza una excepción, entonces `self.map` también lo hará.

**Experimenta** quitando el primer `try` y observa lo que pasa.

In [369]:
do {
     print("kKGRy".toMasterMindColorCombination())
} catch {
    print("Error")
}

: 

### Ejercicio 2: versión inicial de `MasterMindGame`

Con lo anterior ya tenemos los bloques suficientes para empezar a engarzar un juego de MasterMind.

La celda siguiente contiene tan sólo el esqueleto de un `struct` que modela una partida de MasterMind. Tenemos las siguientes variables:
* `secretCode` es una variable privada en la que almacenaremos el código secreto que hay que adivinar.
* `maxTurns` es una constante con el número máximo de turnos, que vamos a fijar en 10.
* `currentTurn` contendrá el número de turno que estamos jugando. La utilizaremos dentro de unas celdas.

In [196]:
struct MasterMindGame {
    private var secretCode: [MasterMindColor]
    
    let maxTurns = 10
    var currentTurn = 0
    
    static func randomCode(colors: Int) -> [MasterMindColor] 
    {

        let arrayletter : [Character] = ["r","g","b","k","y","w"]
//como he creado un array de characters luego genera un error ya que el codigo lo pide como un array de mastermindcolor
        var arraycolor = [Character]()
        for _ in 1...colors 
        {
            arraycolor.append(arrayletter.randomElement()!)       
        }   
        let stringRep = String(arraycolor)

        return try! stringRep.toMasterMindColorCombination()
    }
    
    init(_ secretCode: String? = nil)
    {
         if (secretCode != nil)
        {
           
            guard let code = secretCode else
            {
                self.secretCode = MasterMindGame.randomCode(colors: 4)   
                return
            }

           let hack = try! code.toMasterMindColorCombination()  
            self.secretCode = hack
        }
        else
       {
         self.secretCode =  MasterMindGame.randomCode(colors: 4)    
        }
     
    }
}

In [13]:
struct Juego 
{
  var x :Int = 2
  var y :Int = 3
}

In [14]:
var j = Juego()
print (j)

Juego(x: 2, y: 3)


In [15]:
let juego = MasterMindGame.randomCode(colors: 4)
print(juego)

[🟡, 🔵, 🟢, ⚫]


In [50]:
let prueba1 = MasterMindGame("yrbg")
print(prueba1.secretCode)

[🟡, 🔴, 🔵, 🟢]


Se pide:

1. **En primer lugar, implementa `randomCode`** para obtener una combinación aleatoria del número de posiciones que se le indica en el argumento `colors`. Es decir, si `colors` es 4, esa función devolverá un array de 4 `MasterMindColor` seleccionados de entre todos los posibles.

Para hacer esta función, puedes apoyarte en el método `randomElement()` de la clase `Array`, que lo que hace es obtener un elemento cualquiera del array. Por ejemplo:

In [36]:
let testArray = ["z", "k", "y", "b", "t"]
print(testArray.randomElement())

Optional("y")


**Importante**: `randomElement()` devuelve un opcional, porque podría darse el caso que el array estuviera vacío. Si así fuera, `randomElement()` devolvería `nil`.

Puedes usar o no `randomElement()`, a tu criterio.

2. **En segundo lugar, implementa `init`** para que funcione del siguiente modo:
* Si no se le pasa ningún parámetro, se generará una combinación aleatoria de 4 colores utilizando `randomCode`. Esta será la forma habitual de crear una nueva partida. Observa que `randomCode` es estática, por lo que hay que invocarla utilizando `MasterMindGame.randomCode`.
* Si se le pasa un `String`, `init` intentará convertirlo a un array de colores, utilizando para ello la función `toMasterMindColorCombination()` que hemos definido antes.
  * Si la conversión es correcta, el array de colores será nuestro código secreto.
  * Si la conversión falla, generaremos una combinación aleatoria de 4 colores.
  Esta versión de `init` está pensada para ayudarnos a probar cuando estemos probando el juego. Suministrando una clave conocido podremos verificar más fácilmente las funciones que iremos implementando después.

Las siguientes celdas te ayudarán a verificar si el comportamiento es correcto. Como en el anterior ejercicio, no deben imprimir nada si todo es correcto.

In [190]:
let testGame = MasterMindGame("RGBK")
if testGame.secretCode != [.red, .green, .blue, .black] {
    print("Error al crear una partida con una combinación concreta.")
}

var gamesAreAllTheSame = true
for _ in 0...5 {
    let game1 = MasterMindGame()
    let game2 = MasterMindGame()
  if game1.secretCode != game2.secretCode { gamesAreAllTheSame = false }
}
if gamesAreAllTheSame {
    print("Parece que `randomCode` está generando siempre la misma clave.")
}

//let anotherGame = MasterMindGame("esta cadena no es válida")
//if anotherGame.secretCode.count != 4 {
//    print("La clave aleatoria debe tener 4 colores")
//}

In [303]:
let testGame = MasterMindGame()
print(testGame.secretCode)

[🟡, 🔴, 🟢, ⚪]


In [335]:
let testGame = MasterMindGame("este codigo no es valido") //pasa cuando generas un string que no sea del mismo rango del secretcode
print(testGame.secretCode)

Fatal error: 'try!' expression unexpectedly raised an error: __lldb_expr_21.MasterMindError.wrongCharacter: file __lldb_expr_1755/<Cell 334>, line 32
Current stack trace:
0    libswiftCore.so                    0x00007f04ba8a1f40 swift_reportError + 50
1    libswiftCore.so                    0x00007f04ba90df30 _swift_stdlib_reportFatalErrorInFile + 115
2    libswiftCore.so                    0x00007f04ba577f8e <unavailable> + 1621902
3    libswiftCore.so                    0x00007f04ba575da0 _assertionFailure(_:_:file:line:flags:) + 827
4    libswiftCore.so                    0x00007f04ba5ce820 Dictionary.init<A>(_:uniquingKeysWith:) + 0


: 

## Completando `MasterMindGame`: identificando aciertos y semiaciertos

**Ejercicio 3**

Implementa `countExactMatches`, una función que debes añadir a `MasterMindGame` y que debe cumplir la siguiente especificación:
* Debe aceptar como único argumento un array de `MasterMindColor`, que no tendrá nombre externo.
* Debe devolver un `Int` con el número de aciertos entre la combinación indicada y el código secreto.
* No es necesario verificar que la longitud de la clave coincide con la de la combinación que se suministra (aunque puede hacerse, si se desea).

In [197]:
extension MasterMindGame
{
    func countExactMatches(_ code:[MasterMindColor])-> Int
    {
        var success = 0
        for i in 0...3 
        {
            if secretCode[i] == code [i]
            {
                success = success + 1
            }
        }
        return success
    }
}

Como en los casos anteriores, la siguiente celda mostrará mensajes (o no compilará) si tu implementación no es correcta.

In [198]:
var testGame = MasterMindGame()
if testGame.countExactMatches(testGame.secretCode) != testGame.secretCode.count {
    print("Cuando se suministra la clave secreta, todos los colores deberían coincidir.")
}

func compareMatches(code: String, guess: String, expected: Int) {
    let game = MasterMindGame(code)
    let colors = try! guess.toMasterMindColorCombination()
    let nMatches = game.countExactMatches(colors)
    if expected != nMatches {
        print("La combinación \(guess) debería tener \(expected) aciertos con la clave \(code), pero en tu código salen \(nMatches)")
    }
}

compareMatches(code:"RGGB", guess:"KKKK", expected: 0)
compareMatches(code:"RGGB", guess:"GGGG", expected: 2)
compareMatches(code:"RGGB", guess:"RBBB", expected: 2)
compareMatches(code:"RGGB", guess:"GRRG", expected: 0)
compareMatches(code:"RGGB", guess:"KGBY", expected: 1)
compareMatches(code:"RGGB", guess:"GGBG", expected: 1)


---

**Ejercicio 4**

Implementa `countPartialMatches`, una función que debes añadir a `MasterMindGame`. Debe funcionar de la misma forma que `countExactMatches`, pero contando los _semiaciertos_ en lugar de los aciertos. Revisa [la definición](#Funcionamiento) si es necesario.

In [408]:
extension MasterMindGame
{
    func countPartialMatches(_ code: [MasterMindColor]) -> Int
    {
        var succes = 0
        var successPartial = 0
        // print ("vuelta ")
// se crea un error cuando el acierto parcial esta repetido o existe un acierto
//y el mismo color es un semihacierto lo cuenta 2 veces
        for i in 0...3
        {
           
            if code.contains(secretCode[i]) && secretCode[i] != code[i]
            {
              //print("El color que coincide es: \(secretCode[i])")
              successPartial = successPartial + 1
                    
            }
            
        }
      
        return successPartial
}
}

La siguiente celda mostrará mensajes (o no compilará) si tu implementación no es correcta.

In [409]:
func comparePartialMatches(code: String, guess: String, expected: Int) {
    let game = MasterMindGame(code)
    let colors = try! guess.toMasterMindColorCombination()
    let nMatches = game.countPartialMatches(colors)
    if expected != nMatches {
        print("La combinación \(guess) debería tener \(expected) semiaciertos con la clave \(code), pero en tu código salen \(nMatches)")
    }
}

comparePartialMatches(code:"RGGB", guess:"KKKK", expected: 0)
comparePartialMatches(code:"RGGB", guess:"GGRB", expected: 2)
comparePartialMatches(code:"RGGB", guess:"RBBB", expected: 0)
comparePartialMatches(code:"RGGB", guess:"GRRG", expected: 3)
comparePartialMatches(code:"RGGB", guess:"KGBB", expected: 1)
comparePartialMatches(code:"RGGB", guess:"GGBG", expected: 2)

## Completando `MasterMindGame`: procesando el turno

**Ejercicio 5**

Para terminar la implementación del juego, ya sólo tenemos que implementar una función que gestione un nuevo turno. Completa el esqueleto siguiente de modo que cumpla la siguiente especificación:
* Si la combinación indicada en `guess` no representa una combinación de colores válida, `newTurn` debe imprimir el siguiente mensaje de error y no hacer nada más: `"Combinación incorrecta. Por favor, prueba de nuevo."`.
* Si la combinación tiene un número de colores distinto al del código secreto, `newTurn` debe imprimir el siguiente mensaje de error y no hacer nada más: `"Debes hacer una apuesta con \(secretCode.count) colores. Por favor, prueba de nuevo."`.
* Si se ha alcanzado el número máximo de turnos, o el jugador ha adivinado la clave en un turno anterior, `newTurn` debe imprimir el siguiente mensaje de error y no hacer nada más: `"El juego ha terminado."`.
* En los demás casos, `newTurn` hará lo siguiente:
  - Sumar 1 al número de turno.
  - Mostrar la combinación seleccionada por el usuario.
  - Indicar el número de aciertos y semiaciertos.
  - Si el jugador ha ganado, mostrará el mensaje `Has ganado en el turno x !`, y actualizará el estado de la partida para que no se puedan realizar más turnos.
  - Si el jugador ha perdido, mostrará el mensaje `Lo siento, has perdido. Otra vez será.`.

In [413]:
let MGame = MasterMindGame()
var n = 0
var count = 0
extension MasterMindGame {
    mutating func newTurn(_ guess: String) 
    {
        let turn = try? guess.toMasterMindColorCombination()
        let code = try! guess.toMasterMindColorCombination()
        let success = MGame.countExactMatches(code)
        let seccessPartial = MGame.countPartialMatches(code)
       // print(MGame)
       //para ver el codigo secreto que se genera 
        if guess.count != secretCode.count 
        {
            print("Debes hacer una apuesta con \(secretCode.count) colores. Por favor, prueba de nuevo.")
            return
        }
        
        if turn == nil
        {
            print("Combinación incorrecta. Por favor, prueba de nuevo.")
            return
        }
        else
        {
            count = count + 1
            print("Turno \(count)")
            print("combinacion \(code)")
        }
        
        if count == maxTurns
        {
            print("Lo siento, has perdido. Otra vez será")
            print("El juego a terminado")
            
        }
        
        if code == MGame.secretCode
        {
            if n == 1 
            { 
                print("¡Ya has ganado!. El juego a terminado") 
                return 
            }
            else if n == 0 
            {
                print("¡Has ganado en el turno \(count)!")
                count = count + 1
                return
            }
        }
        print("Aciertos: \(success)")
        print("Semiciertos: \(seccessPartial)")
    }
}

In [416]:
var game3 = MasterMindGame()
game3.newTurn("wgkg")

MasterMindGame(secretCode: [⚪, 🟢, ⚫, 🟢], maxTurns: 10, currentTurn: 0)
Turno 3
combinacion [⚪, 🟢, ⚫, 🟢]
¡Has ganado en el turno 3!


In [149]:
var game2 = MasterMindGame()
game2.newTurn("RGBY")

Turno 13
combinacion [🔴, 🟢, 🔵, 🟡]
Aciertos: 1
Semiciertos: 3


**Ejercicio 6**

En lugar de darte nosotros el código para probar si tu implementación es correcta, en este caso debes crearlo tú mismo. Lo mejor es que leas el enunciado del Ejercicio 5, y después crees el código de prueba utilizando lo que se describe en la especificación. No necesitas que el código funcione para saber cómo tiene que funcionar. Trata de ser lo más exhaustivo posible. Guíate en el código de pruebas que hemos suministrado para los casos anteriores.

In [350]:
func tryGame(code: String, guess: String, expected1: Int, expected2: Int) {
    let game = MasterMindGame(code)
    let colors = try! guess.toMasterMindColorCombination()
    let success = game.countExactMatches(colors)
    let successPartial = game.countPartialMatches(colors)
    if expected1 != success {
        print("La combinación \(guess) debería tener \(expected1) semiaciertos con la clave \(code), pero en tu código salen \(success)")
    }
    if expected2 != successPartial {
        print("La combinación \(guess) debería tener \(expected2) semiaciertos con la clave \(code), pero en tu código salen \(successPartial)")
    }
}

comparePartialMatches(code:"RGGB", guess:"KKKK", expected: 0)
comparePartialMatches(code:"RGGB", guess:"GGGG", expected: 0)
comparePartialMatches(code:"RGGB", guess:"RBBB", expected: 0)
comparePartialMatches(code:"RGGB", guess:"GRRG", expected: 3)
comparePartialMatches(code:"RGGB", guess:"GGBG", expected: 2)

----

A continuación se muestra un ejemplo de una partida completa una vez terminada la implementación:

In [165]:
var testGame = MasterMindGame()

In [166]:
testGame.newTurn("RRBB")

Turno 1
combinacion [🔴, 🔴, 🔵, 🔵]
Aciertos: 0
Semiciertos: 1


In [167]:
testGame.newTurn("RRGG")

Turno 2
combinacion [🔴, 🔴, 🟢, 🟢]
Aciertos: 0
Semiciertos: 0


In [168]:
testGame.newTurn("🔴🔴🔴🔴🔴🔴🔴")

Debes hacer una apuesta con 4 colores. Por favor, prueba de nuevo.


In [169]:
testGame.newTurn("BBKK")

Turno 3
combinacion [🔵, 🔵, ⚫, ⚫]
Aciertos: 1
Semiciertos: 0


In [170]:
testGame.newTurn("RYYR")

Turno 4
combinacion [🔴, 🟡, 🟡, 🔴]
Aciertos: 0
Semiciertos: 1


In [171]:
testGame.newTurn("🔴🟡⚪🔴")

Turno 5
combinacion [🔴, 🟡, ⚪, 🔴]
Aciertos: 1
Semiciertos: 2


In [172]:
testGame.newTurn("🔴🟡⚪🔴")

Turno 6
combinacion [🔴, 🟡, ⚪, 🔴]
Aciertos: 1
Semiciertos: 2
